In [1]:
import numpy as np
import pandas as pd
import nltk
# nltk.download("punkt")
import re
import networkx as nx

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

### Lecture du fichier

In [2]:
df = pd.read_csv("../../files/cgu_figaro.csv", sep="|", header=None, names=["id", "text", "source"])
df.head()

,id,text,source
0,1,Conditions Générales d'Utilisation\n\n \n\nSoc...,http://mentions-legales.lefigaro.fr/page/cgu


In [3]:
sentences = []
for s in df["text"]:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [4]:
sentences[:5]

["Conditions Générales d'Utilisation\n\n \n\nSociété du Figaro, SAS au capital de 16.860.475 €, dont le siège est 14, bd Haussmann, 75009 PARIS, immatriculée au Registre du Commerce et des Sociétés de Paris sous le numéro 542 077 755 (ci-après « Le Figaro »), édite un site d'informations générales www.lefigaro.fr, ci-après le « Site » permettant un accès à différentes informations, ainsi qu'à différents services et produits complémentaires, détaillés ci-après.",
 'Les présentes conditions générales d\'utilisation, ("CGU"), soumises au droit français, ont vocation à régir l\'utilisation du Site ainsi que l\'utilisation des différents services payants ou gratuits (ci-après le(s) « Service(s) »).',
 "La navigation dans le Site et/ou l'inscription à un Service emporte de plein droit acceptation expresse et sans réserve des CGU par l'utilisateur du Site.",
 "En cas de non respect des termes des présentes CGU, Le Figaro se réserve le droit de prendre toute mesure de nature à préserver ses in

In [5]:
word_embeddings = {}
f = open('ressources/multilingual_embeddings.fr', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

40417

In [6]:
word_embeddings[list(word_embeddings)[0]]

array([ 3.6074609e-01, -8.0667698e-01,  8.7549436e-01,  6.2351477e-01,
       -9.2155904e-01,  7.3180795e-01, -2.8121206e-01,  2.9078028e-01,
        2.2967534e+00, -5.4132676e-01, -8.7421370e-01, -2.1018517e+00,
       -3.1696257e-01,  6.7254776e-01,  3.7584463e-01,  1.7269307e-01,
       -2.2942209e-01,  1.0301933e-01, -9.3177575e-01, -5.8207506e-01,
        5.9962851e-01, -4.1104326e-01, -4.8343238e-01,  5.3393024e-01,
        1.7644124e-01, -1.6351806e+00, -2.7169141e-01,  2.2513664e+00,
       -3.8320854e-01,  7.0252633e-01, -3.6192143e-01,  5.2295840e-01,
       -4.7893530e-01,  4.8923010e-01,  8.2576501e-01,  2.4418201e-01,
       -9.2176473e-01, -1.3747742e+00,  9.5153458e-02, -8.0806792e-01,
        1.3131113e+00,  7.5738752e-01,  2.3624134e-01, -5.2581483e-01,
        4.5664132e-01, -1.6880898e+00, -8.1981796e-01,  9.4705112e-02,
        3.3876941e-02,  8.3726294e-02,  6.0565919e-01, -1.2310129e+00,
       -4.9829444e-01,  2.6551306e+00, -1.5972042e+00,  8.7155634e-01,
      

In [7]:
clean_sentences = []
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Zàéêèç]" , " ")
clean_sentences = [sentence.lower() for sentence in clean_sentences]
clean_sentences[:5]

['conditions générales d utilisation     société du figaro  sas au capital de               dont le siège est     bd haussmann        paris  immatriculée au registre du commerce et des sociétés de paris sous le numéro              ci après   le figaro     édite un site d informations générales www lefigaro fr  ci après le   site   permettant un accès à différentes informations  ainsi qu à différents services et produits complémentaires  détaillés ci après ',
 'les présentes conditions générales d utilisation    cgu    soumises au droit français  ont vocation à régir l utilisation du site ainsi que l utilisation des différents services payants ou gratuits  ci après le s    service s     ',
 'la navigation dans le site et ou l inscription à un service emporte de plein droit acceptation expresse et sans réserve des cgu par l utilisateur du site ',
 'en cas de non respect des termes des présentes cgu  le figaro se réserve le droit de prendre toute mesure de nature à préserver ses intérêts 

In [8]:
stop_words = stopwords.words('french')
type(stop_words)
stop_words[:5]

['au', 'aux', 'avec', 'ce', 'ces']

In [9]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[:5]

['conditions générales utilisation société figaro sas capital dont siège bd haussmann paris immatriculée registre commerce sociétés paris sous numéro ci après figaro édite site informations générales www lefigaro fr ci après site permettant accès différentes informations ainsi différents services produits complémentaires détaillés ci après',
 'présentes conditions générales utilisation cgu soumises droit français vocation régir utilisation site ainsi utilisation différents services payants gratuits ci après service',
 'navigation site inscription service emporte plein droit acceptation expresse sans réserve cgu utilisateur site',
 'cas non respect termes présentes cgu figaro réserve droit prendre toute mesure nature préserver intérêts notamment assurer exécution',
 'figaro pourra notamment cette fin assigner utilisateur site responsabilité civile pénale']

In [10]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split()) + 0.001)
    else:
        v =np.zeros((300,))
    sentence_vectors.append(v)
sentence_vectors[:1]

[array([ 1.84877229e-01,  7.47599497e-02, -1.27675818e-01, -4.88224846e-03,
         6.14017753e-02,  8.20970309e-02, -6.28190553e-02, -8.73470142e-02,
         6.44845052e-02,  7.85821026e-01, -2.35702073e-01,  4.63838136e-02,
         9.49847618e-02, -7.34277601e-02, -4.93028050e-02, -4.11498473e-02,
        -1.00786455e-01,  6.42629116e-01, -1.23830651e-01, -1.01940229e-01,
        -1.51724260e-01, -2.52182850e-03, -9.47192002e-02,  5.39087870e-02,
         6.02231503e-02,  1.07721617e-02,  1.03767133e-01, -3.02338018e-02,
        -9.65242665e-02,  5.93051865e-02,  1.51763874e-01, -1.72776404e-02,
        -1.00023277e-02,  1.46311980e-01,  7.31745724e-02, -1.07332764e-01,
        -1.03145372e-01, -2.42834508e-01, -7.94770228e-02, -2.37648272e-02,
         6.42820849e-02,  3.37855611e-02, -8.87708850e-02, -6.21203227e-02,
        -1.04778278e-01,  3.44385959e-02, -4.72998407e-02, -3.23056960e-02,
        -5.47685580e-02, -1.66950587e-01, -3.30785191e-02, -8.49137885e-03,
         1.6

In [11]:
sim_mat = np.zeros([len(sentences), len(sentences)])
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1, 300))[0,0]
sim_mat[1, 2]

0.655685899621368

In [12]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
scores

{0: 0.007690257518884284,
 1: 0.007578908267825406,
 2: 0.00657398021801416,
 3: 0.007426140284274317,
 4: 0.006822664458428999,
 5: 0.00630611225848876,
 6: 0.004900443608161022,
 7: 0.007569000756476756,
 8: 0.0070903236958355185,
 9: 0.007716003341731825,
 10: 0.007347430821910529,
 11: 0.004806905145018013,
 12: 0.008062768153766855,
 13: 0.007911595045275204,
 14: 0.0070606718699543335,
 15: 0.007464841374690148,
 16: 0.007268173571915298,
 17: 0.006390540204948562,
 18: 0.005472736783652032,
 19: 0.006988384292942214,
 20: 0.007833899679379225,
 21: 0.007901793251956588,
 22: 0.00762966319078329,
 23: 0.005616203202779726,
 24: 0.007755988024387561,
 25: 0.006863694862551406,
 26: 0.00808145651561022,
 27: 0.006919381282844434,
 28: 0.004185406778433599,
 29: 0.007770612625315347,
 30: 0.006242832828146442,
 31: 0.007918670217846705,
 32: 0.00704506528320134,
 33: 0.007061833560273904,
 34: 0.0075058866139688304,
 35: 0.0071462547717253124,
 36: 0.007471312490514579,
 37: 0.00723

In [13]:
resultat = ""
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
for i in range(10):
    resultat += ranked_sentences[i][1]

In [14]:
print(resultat)

L'Utilisateur reconnaît et accepte sans réserve être pleinement responsable de toute action entreprise faisant suite à la lecture d'un conseil, d'une opinion, d'un avis d'une cotation financière ou de toutes autres informations présentes dans le Site.Société du Figaro consent à l’Utilisateur du Site un droit d’usage privé, non collectif et non exclusif sur son contenu.. Ce droit est limité au droit d’imprimer un ou plusieurs articles et/ou de l’enregistrer sur son ordinateur (ou autre support digital tel que mobile et tablette) pour son usage personnel exclusivement (y compris dans le cadre d’un abonnement pour les contenus payants).Article 2 - Présentation du Site

Le Site propose notamment à l'Utilisateur :

    la consultation de contenus et d'informations présents sur le Site (moyennant, le cas échéant, la souscription d’un abonnement pour les contenus payants)
    différents Services, sous réserve de son inscription préalable, lui permettant d'avoir accès à un contenu spécifique e

In [17]:
f = open("resultat/resultat_summarization_fr.txt", "w")
f.write(resultat)
f.close()

In [18]:
print(ranked_sentences)

[(0.008083437493714976, "L'Utilisateur reconnaît et accepte sans réserve être pleinement responsable de toute action entreprise faisant suite à la lecture d'un conseil, d'une opinion, d'un avis d'une cotation financière ou de toutes autres informations présentes dans le Site."), (0.00808145651561022, 'Société du Figaro consent à l’Utilisateur du Site un droit d’usage privé, non collectif et non exclusif sur son contenu.. Ce droit est limité au droit d’imprimer un ou plusieurs articles et/ou de l’enregistrer sur son ordinateur (ou autre support digital tel que mobile et tablette) pour son usage personnel exclusivement (y compris dans le cadre d’un abonnement pour les contenus payants).'), (0.008062768153766855, "Article 2 - Présentation du Site\n\nLe Site propose notamment à l'Utilisateur :\n\n    la consultation de contenus et d'informations présents sur le Site (moyennant, le cas échéant, la souscription d’un abonnement pour les contenus payants)\n    différents Services, sous réserve

In [19]:
with open('ressources/multilingual_embeddings.fr', encoding='utf8') as myfile:
    [print(next(myfile)) for x in range(40)]
    print("\n\n\n")

lockheed 0.360746099055 -0.806676984119 0.875494364849 0.623514758276 -0.921559046038 0.731807917957 -0.281212075663 0.290780265834 2.29675344132 -0.541326768881 -0.874213701174 -2.10185174419 -0.316962558753 0.672547728388 0.375844637772 0.172693069249 -0.229422085667 0.103019326859 -0.931775776736 -0.582075036844 0.599628486353 -0.411043262492 -0.483432370748 0.533930217432 0.176441236812 -1.63518056377 -0.271691404601 2.25136632399 -0.383208549455 0.702526357006 -0.361921426281 0.522958396257 -0.478935314565 0.489230099039 0.825765039307 0.24418200263 -0.921764729348 -1.37477423072 0.0951534585281 -0.808067920743 1.31311133024 0.757387498136 0.236241339153 -0.525814806532 0.456641324605 -1.68808983054 -0.819817980335 0.0947051155701 0.0338769393039 0.0837262963933 0.605659210465 -1.23101289047 -0.498294431089 2.65513055271 -1.59720423859 0.871556344049 1.44021853606 0.204658287983 -2.05402603589 -0.572922392495 0.791747058532 -0.750468724576 0.955419144502 0.896903707575 -0.10619111

In [17]:
import numpy as np

toto = {}
word="test"
coefs = np.asarray([0.153, 0.185, 0,213], dtype='float32')
toto[word] = coefs

In [18]:
toto[list(toto)[0]]

array([1.53e-01, 1.85e-01, 0.00e+00, 2.13e+02], dtype=float32)

In [23]:
toto.get("toto", np.zeros((100,)))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])